In [ ]:
# https://www.pinecone.io/learn/data-augmentation/ augmentedSBERT
# https://github.com/snunlp/KR-SBERT 한국어 SBERT
# from sentence_transformers import SentenceTransformer, util
# model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

In [ ]:
# sentences = ['잠이 옵니다', '졸음이 옵니다', '기차가 옵니다']
# vectors = model.encode(sentences) # encode sentences into vectors
# print(vectors.shape)
# similarities = util.cos_sim(vectors, vectors) # compute similarity between sentence vectors
# print(similarities)

문장을 벡터화해서 문장 간 유사도를 구하는 모델.<br>
그러면 이걸 가지고 multi-label 분류를 할 수도 있지 않을까?<br>
벡터화를 일단 시키고 그걸 인공신경망에 넣어서 11개의 태스크로 분류하기(multi-label)

In [1]:
!pip install scikit-multilearn
!pip install sentence_transformers
!pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 KB 3.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 34.0 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=02280dffcf6b25b873c86ba6f4335f8cd127001cd2e3ab42d62b56ebac36348a
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54a

In [2]:
# import
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from skmultilearn.model_selection import iterative_train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from keras import backend as K
from sentence_transformers import SentenceTransformer, util, models, losses
import tensorflow as tf
import tensorflow_addons as tfa
# import wandb
import torch
from tensorflow.keras.metrics import BinaryAccuracy, AUC
from tensorflow_addons.metrics import F1Score
from keras.regularizers import l1, l2
from sentence_transformers.readers import InputExample
from torch.utils.data import DataLoader
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
import math

In [ ]:
# wandb.init(project='cp2_s_bert')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nemyrmain. Use `wandb login --relogin` to force relogin


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
filepath = '/content/drive/MyDrive/review_w_only_labels (3).csv'
df = pd.read_csv(filepath)
columns = df.columns[1:].tolist()
columns.remove('review_tags')
columns.remove('target')
df = df[columns]
df

,review,가성비,귀여운,넓은,단체,만족,모던,분위기,비주얼,아늑,위생,응대,이색음식,이색테마,클래식,혼자
0,고기도 구워주시고 친절도 하시고 맛도 최고입니다.,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,가게 인기가 좋아서 그런지 가게가 바빴는데 직원분들 저너무 친절하시구 초벌 도와주신...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,공간도 깔끔하고 맛있습니다.,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,웨이팅은 했지만 깔끔하고 송년모임 하기 좋은것같아요 다음에 올때는 6시전에 좀 일찍...,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
4,직원분들이 친절하고 좋아용.리뷰만 보고 방문했는데 이렇게 분위기 좋은 고깃집은 첨이...,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2609,"치즈가 쭉쭉 늘어날 정도로 많이 들어있고, 튀김은 엄청 바삭하고, 김밥은 너무 맛있...",1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2610,양많고 맛나요.치즈불맛덕에 더 특별한 맛.테이블은 조금 좁아요,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2611,치즈 폭포 진짜 갬동 그자체,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2612,떡볶이중독자라 인스타에서 우연히 보고 먹으러왔어요 매운떡볶이를 좋아하지만 치즈가 가...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [5]:
targets = df.columns.tolist()[1:]
targets

['가성비',
 '귀여운',
 '넓은',
 '단체',
 '만족',
 '모던',
 '분위기',
 '비주얼',
 '아늑',
 '위생',
 '응대',
 '이색음식',
 '이색테마',
 '클래식',
 '혼자']

In [6]:
X = np.array(pd.DataFrame(df['review']).reset_index())
y = np.array(df[targets])

In [7]:
np.random.seed(42)
X_train, y_train, X_test, y_test = iterative_train_test_split(X, y, test_size=0.3)

In [8]:
train_index = X_train[:, 0]
test_index = X_test[:, 0]
len(train_index) + len(test_index)

2614

In [9]:
df_melt = df.loc[train_index, :]
df_melt = pd.melt(df_melt.reset_index(), id_vars=['index', 'review'])
df_melt

,index,review,variable,value
0,1,가게 인기가 좋아서 그런지 가게가 바빴는데 직원분들 저너무 친절하시구 초벌 도와주신...,가성비,0
1,2,공간도 깔끔하고 맛있습니다.,가성비,0
2,3,웨이팅은 했지만 깔끔하고 송년모임 하기 좋은것같아요 다음에 올때는 6시전에 좀 일찍...,가성비,0
3,6,생갈비 굳굳.사장님 완전 친절하시고 설명도 자세하게 잘해주세요,가성비,0
4,7,사누끼 우동이 아주 일품입니다.재방문 의사 100.맛있습니다.추천,가성비,0
...,...,...,...,...
27535,2609,"치즈가 쭉쭉 늘어날 정도로 많이 들어있고, 튀김은 엄청 바삭하고, 김밥은 너무 맛있...",혼자,0
27536,2610,양많고 맛나요.치즈불맛덕에 더 특별한 맛.테이블은 조금 좁아요,혼자,0
27537,2611,치즈 폭포 진짜 갬동 그자체,혼자,0
27538,2612,떡볶이중독자라 인스타에서 우연히 보고 먹으러왔어요 매운떡볶이를 좋아하지만 치즈가 가...,혼자,0


SBERT 모델 train

In [10]:
s_bert_path = '/content/drive/MyDrive/cp2/s_bert_output'
# word_embedding_model = models.Transformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
# pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
# dense_model = models.Dense(in_features=pooling_model.get_sentence_embedding_dimension(), out_features=512, activation_function=torch.nn.Tanh())

# s_bert = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model])
s_bert = SentenceTransformer(s_bert_path) # 학습 완료되면 이걸로 쓰기

In [ ]:
num_epochs = 1
batch_size = 32

df_gold = df_melt.reset_index(drop=True)
gold_samples = []
dev_samples = []

for i in range(len(df_gold)):
    gold_samples.append(InputExample(texts=[df_gold.loc[i, 'review'], df_gold.loc[i, 'variable']], label=float(df_gold.loc[i, 'value'])))
train_dataloader = DataLoader(gold_samples, shuffle=True, batch_size=batch_size)
train_loss = losses.CosineSimilarityLoss(s_bert)

# configure the training
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1)  # 10% of train data

In [28]:
s_bert.fit(train_objectives=[(train_dataloader, train_loss)],
          # evaluator=evaluator,
          epochs=num_epochs,
          # evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=s_bert_path
          )

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/860 [00:00<?, ?it/s]

Iteration:   0%|          | 0/860 [00:00<?, ?it/s]

Iteration:   0%|          | 0/860 [00:00<?, ?it/s]

Iteration:   0%|          | 0/860 [00:00<?, ?it/s]

Iteration:   0%|          | 0/860 [00:00<?, ?it/s]

문장 벡터화

In [11]:
# X = df['review']
# index = X.index
# vectors = s_bert.encode(X.tolist())
# vectors = pd.DataFrame(vectors)
# vectors.index = index
# vectors
X_train = X_train[:, 1]
X_test = X_test[:, 1]
vectors_train = s_bert.encode(X_train.tolist())
vectors_test = s_bert.encode(X_test.tolist())

In [12]:
y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

Sequential 모델 train

In [ ]:
# _vectors = np.array(vectors)
# _y = np.array(y)
# columns = y.columns

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(vectors, y, test_size=0.3, random_state=42, stratify=y)
# from skmultilearn.model_selection import iterative_train_test_split
# X_train, y_train, X_test, y_test = iterative_train_test_split(_vectors, _y, test_size = 0.4)    # pandas.DataFrame이 아니라 np.array로 넣어야 함

In [13]:
df_y_train = pd.DataFrame(y_train)
df_y_train.columns = targets

In [14]:
class_weights = df_y_train.sum() / (len(df_y_train) - df_y_train.sum())  # 1인 개수 / 0인 개수
class_weights = 1 / class_weights   # (체크) 이거인가?
class_weights_list = class_weights.tolist()
class_weights = class_weights.to_dict()
i = 0
keys = list(class_weights.keys())
for k in keys:
    class_weights[i] = class_weights.pop(k)
    i += 1
class_weights

{0: 6.524590163934426,
 1: 30.118644067796613,
 2: 19.400000000000002,
 3: 67.0,
 4: 8.924324324324324,
 5: 15.392857142857144,
 6: 2.6356435643564353,
 7: 30.655172413793103,
 8: 7.539534883720931,
 9: 8.663157894736843,
 10: 1.5678321678321678,
 11: 9.551724137931034,
 12: 15.392857142857144,
 13: 21.666666666666664,
 14: 24.150684931506852}

In [15]:
X_train = vectors_train
X_test = vectors_test

In [16]:
from keras.regularizers import l1, l2, l1_l2

_accuracy = BinaryAccuracy()
_f1_score = F1Score(num_classes=15)
_auc = AUC(curve='PR', multi_label=True, num_labels=15)

# parameters
epochs = 150
batch_size = 64
# wandb.config = {
#   "epochs": epochs,
#   "batch_size": batch_size
# }

# tf.random.set_seed(42)
model = Sequential([
    Dense(256, activation='relu', kernel_regularizer=l1_l2(0.0001, 0.0001)),
    Dropout(0.4),
    Dense(128, activation='relu', kernel_regularizer=l1_l2(0.0001, 0.0001)),
    Dropout(0.2),
    Dense(15, activation='sigmoid')
])
optimizer = keras.optimizers.Adam(learning_rate=0.0005)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=[_f1_score, _auc])
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, class_weight=class_weights)

loss, f1_score, pr_auc = model.evaluate(X_test, y_test, batch_size=32)
print('Loss: ', loss)
print('f1_score: ', f1_score)
print('pr_auc: ', pr_auc)
f1_score_df = pd.DataFrame()
f1_score_df.index = targets
f1_score_df['f1_score'] = f1_score
f1_score_df

Epoch 1/150
29/29 [==============================] - 2s 5ms/step - loss: 4.1365 - f1_score: 0.3696 - auc: 0.2955
Epoch 2/150
29/29 [==============================] - 0s 5ms/step - loss: 2.2640 - f1_score: 0.6498 - auc: 0.7443
Epoch 3/150
29/29 [==============================] - 0s 5ms/step - loss: 1.7932 - f1_score: 0.6833 - auc: 0.8388
Epoch 4/150
29/29 [==============================] - 0s 6ms/step - loss: 1.5739 - f1_score: 0.6979 - auc: 0.8881
Epoch 5/150
29/29 [==============================] - 0s 5ms/step - loss: 1.5007 - f1_score: 0.7137 - auc: 0.9042
Epoch 6/150
29/29 [==============================] - 0s 5ms/step - loss: 1.4338 - f1_score: 0.7089 - auc: 0.9129
Epoch 7/150
29/29 [==============================] - 0s 5ms/step - loss: 1.3435 - f1_score: 0.7216 - auc: 0.9304
Epoch 8/150
29/29 [==============================] - 0s 5ms/step - loss: 1.3260 - f1_score: 0.7263 - auc: 0.9278
Epoch 9/150
29/29 [==============================] - 0s 5ms/step - loss: 1.2838 - f1_score: 0.73

,f1_score
가성비,0.757062
귀여운,0.808511
넓은,0.500000
단체,0.600000
만족,0.487805
모던,0.303030
분위기,0.732782
비주얼,0.388889
아늑,0.653333
위생,0.671642


In [ ]:
targets

['가성비',
 '귀여운',
 '넓은',
 '단체',
 '만족',
 '모던',
 '분위기',
 '비주얼',
 '아늑',
 '위생',
 '응대',
 '이색음식',
 '이색테마',
 '클래식',
 '혼자']

In [35]:
input = '''뭔가 이상한데..'''
def output(input, model):
    vector = s_bert.encode(input)
    vector = pd.DataFrame(vector).T
    return model.predict(vector)[0].tolist()
def labeling(input):
    return [1 if i > 0.5 else 0 for i in input]
temp = output(input, model)
labeled_temp = labeling(temp)
print([targets[i] for i in range(len(labeled_temp)) if labeled_temp[i]==1])
df_result = pd.DataFrame()
df_result.index = targets
df_result['점수'] = temp
df_result

1/1 [==============================] - 0s 30ms/step
['이색음식', '이색테마']


,점수
가성비,0.002387
귀여운,0.002313
넓은,0.000104
단체,0.019234
만족,0.007817
모던,0.000282
분위기,0.052565
비주얼,0.007148
아늑,0.023909
위생,0.004665


In [ ]:
# save the model
# model.save('s_bert_model.h5', save_format='h5')
# new_model = keras.models.load_model('s_bert_model.h5')

In [ ]:
# save only model weights(not optimizer, architecture...)
# model.save_weights('s_bert_model_parameters.h5')


In [ ]:
# new model test
# input = '''분위기 좋은'''
# temp = output(input, new_model)
# labeled_temp = labeling(temp)
# print([targets[i] for i in range(len(labeled_temp)) if labeled_temp[i]==1])
# df_result = pd.DataFrame()
# df_result.index = targets
# df_result['점수'] = temp
# df_result

In [36]:
class CombinedModel(Sequential):
    def __init__(self, s_bert, model, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.s_bert = s_bert
        self.model = model

    # def build(self, input_shape):
    #     super().build(input_shape)
    
    def predict(self, input):
        vector = self.s_bert.encode(input)
        vector = pd.DataFrame(vector).T
        return self.model.predict(vector)[0].tolist()

combined_model = CombinedModel(s_bert, model)


In [37]:
input = '넓고 매장이 깨끗한 맛집'
print(combined_model.predict(input))

1/1 [==============================] - 0s 44ms/step
[6.876949191791937e-05, 7.217045094876084e-07, 0.9991505146026611, 1.655395135458093e-05, 5.07535005453974e-05, 9.927411883836612e-05, 5.568357300944626e-05, 1.9485260054352693e-05, 1.0098584652951104e-06, 0.9930343627929688, 0.00029893784085288644, 6.477822807937628e-06, 8.216130481741857e-06, 1.6679792906870716e-06, 2.647314249770716e-05]


In [ ]:
# input_shape = (None, 768)
# _ = combined_model.build(input_shape)
# combined_model.save('combined_model')

In [ ]:
# temp_dir = 'C:\AI_bootcamp_python\CP2/bert_model'
# torch.save(s_bert.state_dict(), temp_dir + '/model.bin')

In [ ]:
# model.save('sequential_model')

In [ ]:
# very_new_model = keras.models.load_model('sequential_model')

In [ ]:
# new_combined_model = CombinedModel(s_bert, very_new_model)
# new_combined_model.predict('가성비')

In [ ]:
# s_bert_new = SentenceTransformer('C:\AI_bootcamp_python\CP2/bert_model/model.bin')
# s_bert_new.model = SentenceTransformer.SentenceTransformerModel.from_pretrained(temp_dir)
# s_bert_new.model.load_state_dict(torch.load(temp_dir + '/model.bin'))

In [ ]:
# # pickling the model
# import pickle

# with open('combined_model.pkl', 'wb') as file:
#     pickle.dump(combined_model, file)